## How-to guide for Related Items use-case on Abacus.AI platform
This notebook provides you with a hands on environment to build a model that suggests related items using the Abacus.AI Python Client Library.

We'll be using the [User Item Recommendations](https://s3.amazonaws.com//realityengines.exampledatasets/user_recommendations/user_movie_ratings.csv), [Movie Attributes](https://s3.amazonaws.com//realityengines.exampledatasets/user_recommendations/movies_metadata.csv), and [User Attributes](https://s3.amazonaws.com//realityengines.exampledatasets/user_recommendations/users_metadata.csv) datasets, each of which has information about the user and/or their choice of movies.

1. Install the Abacus.AI library.

In [ ]:
!pip install abacusai

We'll also import pandas and pprint tools for visualization in this notebook.

In [ ]:
import pandas as pd # A tool we'll use to download and preview CSV files
import pprint # A tool to pretty print dictionary outputs
pp = pprint.PrettyPrinter(indent=2)

2. Add your Abacus.AI [API Key](https://abacus.ai/app/profile/apikey) generated using the API dashboard as follows:

In [ ]:
#@title Abacus.AI API Key

api_key = ''  #@param {type: "string"}

3. Import the Abacus.AI library and instantiate a client.

In [ ]:
from abacusai import ApiClient
client = ApiClient(api_key)

## 1. Create a Project

Abacus.AI projects are containers that have datasets and trained models. By specifying a business **Use Case**, Abacus.AI tailors the deep learning algorithms to produce the best performing model possible for your data.

We'll call the `list_use_cases` method to retrieve a list of the available Use Cases currently available on the Abacus.AI platform.

In [ ]:
client.list_use_cases()

In this notebook, we're going to create a model that suggests related items using the User Item Recommendations, Movie Attributes, and User Attributes datasets. The 'USER_RELATED' use case is best tailored for this situation. For the purpose of taking an example, we will use the IMDB movie dataset that has movie metadata, user metadata, and user-movie ratings.

In [ ]:
#@title Abacus.AI Use Case

use_case = 'USER_RELATED'  #@param {type: "string"}

By calling the `describe_use_case_requirements` method we can view what datasets are required for this use_case.

In [ ]:
for requirement in client.describe_use_case_requirements(use_case):
  pp.pprint(requirement.to_dict())

Finally, let's create the project.

In [ ]:
related_items_project = client.create_project(name='Related Movies', use_case=use_case)
related_items_project.to_dict()

**Note: When feature_groups_enabled is False then the use case does not support feature groups (collection of ML features). Therefore, Datasets are created at the organization level and tied to a project to further use them for training ML models**

## 2. Add Datasets to your Project

Abacus.AI can read datasets directly from `AWS S3` or `Google Cloud Storage` buckets, otherwise you can also directly upload and store your datasets with Abacus.AI. For this notebook, we will have Abacus.AI read the datasets directly from a public S3 bucket's location.

We are using three datasets for this notebook. We'll tell Abacus.AI how the datasets should be used when creating it by tagging each dataset with a special Abacus.AI **Dataset Type**.
- [User Item Recommendations](https://s3.amazonaws.com//realityengines.exampledatasets/user_recommendations/user_movie_ratings.csv) (**USER_ITEM_INTERACTIONS**): 
This dataset contains information about multiple users' ratings of movies with specified IDs.
- [Movie Attributes](https://s3.amazonaws.com//realityengines.exampledatasets/user_recommendations/movies_metadata.csv) (**CATALOG_ATTRIBUTES**): This dataset contains attributes about movies with specified IDs, such as each movie's name and genre.
- [User Attributes](https://s3.amazonaws.com//realityengines.exampledatasets/user_recommendations/users_metadata.csv) (**USER_ATTRIBUTES**): This dataset contains information about users with specified IDs, such as their age, gender, occupation, and zip code. 

### Add the datasets to Abacus.AI

First we'll use Pandas to preview the files, then add them to Abacus.AI.

In [ ]:
pd.read_csv('https://s3.amazonaws.com//realityengines.exampledatasets/user_recommendations/user_movie_ratings.csv')

In [ ]:
pd.read_csv('https://s3.amazonaws.com//realityengines.exampledatasets/user_recommendations/movies_metadata.csv')

In [ ]:
pd.read_csv('https://s3.amazonaws.com//realityengines.exampledatasets/user_recommendations/users_metadata.csv')

Using the Create Dataset API, we can tell Abacus.AI the public S3 URI of where to find the datasets. We will also give each dataset a Refresh Schedule, which tells Abacus.AI when it should refresh the dataset (take an updated/latest copy of the dataset).

If you're unfamiliar with Cron Syntax, Crontab Guru can help translate the syntax back into natural language: [https://crontab.guru/#0_12_\*_\*_\*](https://crontab.guru/#0_12_*_*_*)

**Note: This cron string will be evaluated in UTC time zone**

In [ ]:
user_item_dataset = client.create_dataset_from_file_connector(name='User Item Recommendations', table_name='User_Item_Recommendations',
                                     location='s3://realityengines.exampledatasets/user_recommendations/user_movie_ratings.csv',
                                     refresh_schedule='0 12 * * *')

movie_attributes_dataset = client.create_dataset_from_file_connector(name='Movie Attributes', table_name='Movie_Attributes',
                                     location='s3://realityengines.exampledatasets/user_recommendations/movies_metadata.csv',
                                     refresh_schedule='0 12 * * *')

user_attributes_dataset = client.create_dataset_from_file_connector(name='User Attributes', table_name='User_Attributes',
                                     location='s3://realityengines.exampledatasets/user_recommendations/users_metadata.csv',
                                     refresh_schedule='0 12 * * *')

datasets = [user_item_dataset, movie_attributes_dataset, user_attributes_dataset]

for dataset in datasets:
    dataset.wait_for_inspection()

## 3. Create Feature Groups and add them to your Project

Datasets are created at the organization level and can be used to create feature groups as follows:

In [ ]:
feature_group = client.create_feature_group(table_name='Related_Items1',sql='select * from User_Item_Recommendations')

Adding Feature Group to the project:

In [ ]:
client.add_feature_group_to_project(feature_group_id=feature_group.feature_group_id,project_id = related_items_project.project_id)

Setting the Feature Group type according to the use case requirements:

In [ ]:
client.set_feature_group_type(feature_group_id=feature_group.feature_group_id, project_id = related_items_project.project_id, feature_group_type= "USER_ITEM_INTERACTIONS")

Check current Feature Group schema:

In [ ]:
client.get_feature_group_schema(feature_group_id=feature_group.feature_group_id)

#### For each **Use Case**, there are special **Column Mappings** that must be applied to a column to fulfill use case requirements. We can find the list of available **Column Mappings** by calling the *Describe Use Case Requirements* API:

In [ ]:
client.describe_use_case_requirements(use_case)[0].allowed_feature_mappings

In [ ]:
client.set_feature_mapping(project_id=related_items_project.project_id, feature_group_id= feature_group.feature_group_id, feature_name='movie_id', feature_mapping='ITEM_ID')
client.set_feature_mapping(project_id=related_items_project.project_id, feature_group_id= feature_group.feature_group_id,feature_name='user_id', feature_mapping='USER_ID')
client.set_feature_mapping(project_id=related_items_project.project_id, feature_group_id= feature_group.feature_group_id,feature_name='timestamp', feature_mapping='TIMESTAMP')

For each required Feature Group Type within the use case, you must assign the Feature group to be used for training the model:

In [ ]:
client.use_feature_group_for_training(project_id=related_items_project.project_id, feature_group_id=feature_group.feature_group_id)

Now that we've our feature groups assigned, we're almost ready to train a model!

To be sure that our project is ready to go, let's call project.validate to confirm that all the project requirements have been met:

In [ ]:
related_items_project.validate()

## 4. Train a Model

For each **Use Case**, Abacus.AI has a bunch of options for training. We can call the *Get Training Config Options* API to see the available options.

In [ ]:
related_items_project.get_training_config_options()

In this notebook, we'll just train with the default options, but definitely feel free to experiment, especially if you have familiarity with Machine Learning.

In [ ]:
related_items_model = related_items_project.train_model(training_config={})
related_items_model.to_dict()

After we start training the model, we can call this blocking call that routinely checks the status of the model until it is trained and evaluated:

In [ ]:
related_items_model.wait_for_evaluation()

**Note that model training might take some minutes to some hours depending upon the size of datasets, complexity of the models being trained and a variety of other factors**

## **Checkpoint** [Optional]
As model training can take an hours to complete, your page could time out or you might end up hitting the refresh button, this section helps you restore your progress:

In [ ]:
!pip install abacusai
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=2)
api_key = ''  #@param {type: "string"}
from abacusai import ApiClient
client = ApiClient(api_key)
related_items_project = next(project for project in client.list_projects() if project.name == 'Related Movies')
related_items_model = related_items_project.list_models()[-1]
related_items_model.wait_for_evaluation()

## Evaluate your Model Metrics

After your model is done training you can inspect the model's quality by reviewing the model's metrics:

In [ ]:
pp.pprint(related_items_model.get_metrics().to_dict())

To get a better understanding on what these metrics mean, visit our [documentation](https://abacus.ai/app/help/useCases/USER_RELATED/training) page.

## 5. Deploy Model

After the model has been trained, we need to deploy the model to be able to start making predictions. Deploying a model will reserve cloud resources to host the model for Realtime and/or batch predictions.

In [ ]:
related_items_deployment = client.create_deployment(name='Related Items Deployment',description='Related Items Deployment',model_id=related_items_model.model_id)
related_items_deployment.wait_for_deployment()

After the model is deployed, we need to create a deployment token for authenticating prediction requests. This token is only authorized to predict on deployments in this project, so it's safe to embed this token inside of a user-facing application or website.

In [ ]:
deployment_token = related_items_project.create_deployment_token().deployment_token
deployment_token

## 6. Predict


Now that you have an active deployment and a deployment token to authenticate requests, you can make the `get_related_items` API call below.

This command will return a list of related items based on the provided user_id (1) and movie_id (466). The related items list would be determined based on what movies the user liked in the past and how the movies and users are related to each other depending on their attributes.



In [ ]:
ApiClient().get_related_items(deployment_token=deployment_token,
               deployment_id=related_items_deployment.deployment_id,
               query_data={"user_id":"1","movie_id":"466"})